In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline
from scipy import optimize

from tyssue.core.sheet import Sheet
from tyssue.config.json_parser import load_default

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.dynamics.apoptosis_model import SheetApoptosisModel as model
from tyssue.dynamics.sheet_vertex_model import SheetModel as basemodel
from tyssue.solvers.sheet_vertex_solver import Solver as solver

#from tyssue.dynamics.sheet_isotropic_model import isotropic_relax

from tyssue.draw.vispy_draw import vp_view
from tyssue.topology.sheet_topology import type1_transition, remove_face

from tyssue.draw.mpl_draw import sheet_view
import tyssue.draw.mpl_draw as draw
from tyssue.stores import load_datasets

In [23]:
h5store = 'small_hexagonal.hf5'
datasets = load_datasets(h5store,
                         data_names=['face', 'vert', 'edge'])


specs = {
    'edge': {
        'cell': 0,
        'dx': 0.0,
        'dy': 0.0,
        'dz': 0.0,
        'face': 0,
        'length': 0.0,
        'nx': 0.0,
        'ny': 0.0,
        'nz': 1.0,
        'srce': 0,
        'trgt': 0,
        'is_apical': True,
        'is_sagital': False,
        'is_basal': False,
        'sub_area': 0.0,
        },
    'face': {
        'area': 1.0,
        'is_alive': True,
        'num_sides': 6,
        'perimeter': 1.0,
        'x': 0.0,
        'y': 0.0,
        'z': 0.0,
        },
    'cell': {
        'vol': 1.0,
        'is_alive': True,
        'num_sides': 6,
        'perimeter': 1.0,
        'x': 0.0,
        'y': 0.0,
        'z': 0.0,
        },
     'settings': {
        'geometry': 'cylindrical',
        'height_axis': 'z'
        },
     'vert': {
        'basal_shift': 4.0,
        'is_active': True,
        'is_basal': False,
        'rho': 0.0,
        'x': 0.0,
        'y': 0.0,
        'z': 0.0
        }
    }




In [24]:
sheet_specs

{'edge': {'dx': 0.0,
  'dy': 0.0,
  'dz': 0.0,
  'face': 0,
  'length': 0.0,
  'nx': 0.0,
  'ny': 0.0,
  'nz': 1.0,
  'srce': 0,
  'trgt': 0},
 'face': {'area': 1.0,
  'is_alive': True,
  'num_sides': 6,
  'perimeter': 1.0,
  'x': 0.0,
  'y': 0.0,
  'z': 0.0},
 'settings': {'geometry': 'cylindrical', 'height_axis': 'z'},
 'vert': {'basal_shift': 4.0,
  'is_active': True,
  'rho': 0.0,
  'x': 0.0,
  'y': 0.0,
  'z': 0.0}}